In [1]:
#import sys
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
cwd=os.getcwd() #'/home/raghavendr48edu' This is path for instructor-/mnt/home/edureka_121039
#cwd
'''for part in cwd.split('/'):
    if part.lower().startswith('home'):
        user_id=part.title()
user_id''' # user_id= Edureka_121039
for part in cwd.split('/'):
    if part.lower().startswith('ragh'):
        user_id=part.title()
#user_id #'Raghavendr48Edu'

app_name= '{0}:apple_logs'.format(user_id) 
#app_name #'Raghavendr48Edu: RDD HandsOn'

conf=SparkConf().setAppName(app_name)

# This applicationId goes i/p to SparkConfiguration
#sc.applicationId  #'local-1683463288956'

def get_hdfs_filepath(file_name):
    my_hdfs='/user/{0}'.format(user_id.lower())
    return os.path.join(my_hdfs,file_name)

spark=SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
data=spark.read.text(get_hdfs_filepath("access.clean.log"))
#data=spark.read.text("/user/edureka_960126/access.clean.log")
data.show(5,truncate=False)
data.printSchema()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] GET /administrator/ HTTP/1.1 200 4263 - Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 -                                                 |
|109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] POST /administrator/index.php HTTP/1.1 200 4494 http://almhuette-raith.at/administrator/ Mozilla/5.0 (Windows NT 6.0; 

In [4]:
import re
url='83.167.113.100 - - [12/Dec/2015:18:31:25 +0100] GET /administrator/ HTTP/1.1 200 4263 - Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0 - '
host_pattern = '(^\S+\.[\S+\.]+[\S+\.]+\S)+\s'
re.search(host_pattern,url),re.search(host_pattern,url).group()


(<re.Match object; span=(0, 15), match='83.167.113.100 '>, '83.167.113.100 ')

In [5]:
from pyspark.sql.functions import trim

from pyspark.sql.functions import regexp_extract

host_pattern = '(^\S+\.[\S+\.]+[\S+\.]+\S)+\s'
datetime_pattern='\[(\d+/[A-Z][a-z]{2}/[0-9]{4}:[0-9]{2}:[0-9]{2}:[0-9]{2}\s\+\d{4})]'
method='(GET | POST)\s*'
method_url=']\s*(GET | POST)\s*(/\S*)\s*HTTP/\S*\s([0-9]{3})\s([0-9]*)'

logs_df = data.select(regexp_extract('value', host_pattern, 1).alias('host'),
                         regexp_extract('value',datetime_pattern , 1).alias('timestamp'),
                         regexp_extract('value', method_url , 1).alias('method'),
                         regexp_extract('value', method_url, 2).alias('endpoint'),
                         regexp_extract('value', method_url, 3).cast('integer').alias('status'),
                         regexp_extract('value', method_url, 4).cast('integer').alias('content_size'))

logs_df.show(10, truncate=False)

logs_df=logs_df.withColumn("method",trim(logs_df["method"]))

print((logs_df.count(), len(logs_df.columns)))

+---------------+--------------------------+------+------------------------+------+------------+
|host           |timestamp                 |method|endpoint                |status|content_size|
+---------------+--------------------------+------+------------------------+------+------------+
|109.169.248.247|12/Dec/2015:18:25:11 +0100|GET   |/administrator/         |200   |4263        |
|109.169.248.247|12/Dec/2015:18:25:11 +0100| POST |/administrator/index.php|200   |4494        |
|46.72.177.4    |12/Dec/2015:18:31:08 +0100|GET   |/administrator/         |200   |4263        |
|46.72.177.4    |12/Dec/2015:18:31:08 +0100| POST |/administrator/index.php|200   |4494        |
|83.167.113.100 |12/Dec/2015:18:31:25 +0100|GET   |/administrator/         |200   |4263        |
|83.167.113.100 |12/Dec/2015:18:31:25 +0100| POST |/administrator/index.php|200   |4494        |
|95.29.198.15   |12/Dec/2015:18:32:10 +0100|GET   |/administrator/         |200   |4263        |
|95.29.198.15   |12/Dec/2015:1

#### 2 Find out how many 404 HTTP codes are in access logs. 

In [6]:
logs_df.filter("status=404").count()

186787

In [7]:
logs_df.createOrReplaceTempView("logs")

spark.sql('Select count(*) as count_404 From logs where status=404').show(truncate=False)

+---------+
|count_404|
+---------+
|186787   |
+---------+



#### 3 Find out which URLs are broken. 

In [8]:
logs_df.filter("status=404" or "status=400").select('endpoint').take(10)

[Row(endpoint='/templates/_system/css/general.css'),
 Row(endpoint='/templates/_system/css/general.css'),
 Row(endpoint='/favicon.ico'),
 Row(endpoint='/icons/text.gif'),
 Row(endpoint='/templates/_system/css/general.css'),
 Row(endpoint='/apache-log/error.log.44.gz'),
 Row(endpoint='/apache-log/access.log.69.gz'),
 Row(endpoint='/apache-log/error.log.55.gz'),
 Row(endpoint='/templates/_system/css/general.css'),
 Row(endpoint='/favicon.ico')]

In [9]:


#Here are some examples of error codes that a web server may present for a broken link:
#404 Page Not Found: the page/resource doesn’t exist on the server
#400 Bad Request: the host server cannot understand the URL on your page
#Bad host: Invalid host name: the server with that name doesn’t exist or is unreachable
#Bad URL: Malformed URL (e.g. a missing bracket, extra slashes, wrong protocol, etc.)
#Bad Code: Invalid HTTP response code: the server response violates HTTP spec
#Empty: the host server returns “empty” responses with no content and no response code
#Timeout: Timeout: HTTP requests constantly timed out during the link check
#Reset: the host server drops connections. It is either misconfigured or too busy


spark.sql('Select endpoint  From logs where  status=404 or status=400 ').show(truncate=False)

+----------------------------------+
|endpoint                          |
+----------------------------------+
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/favicon.ico                      |
|/icons/text.gif                   |
|/templates/_system/css/general.css|
|/apache-log/error.log.44.gz       |
|/apache-log/access.log.69.gz      |
|/apache-log/error.log.55.gz       |
|/templates/_system/css/general.css|
|/favicon.ico                      |
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/favicon.ico                      |
|/favicon.ico                      |
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
|/templates/_system/css/general.css|
+----------------------------------+
only showing top 20 rows



### 4.Verify there are no null columns in the original dataset. 

In [10]:


print(data.filter(data['value'].isNull()).count())


#so there were no null rows in our data which we loaded from the access logs text file

#Let us check the null values in our regex parsed to check any null columns 

logs_df.filter(logs_df['content_size'].isNull()).count(),logs_df.filter(logs_df['content_size']==0).count()

#so the count below shows that there are the null values spread in our various columns as 66366 rows are having null values

0


(66366, 324)

#### Count of Nulls

In [11]:
#check column wise null values

from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum


# Build up a list of column expressions, one per column.
exprs = [spark_sum(col(col_name).isNull().cast('integer')).alias('countof'+col_name) for col_name in logs_df.columns]
print(exprs)

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
logs_df.agg(*exprs).show()

[Column<b'sum(CAST((host IS NULL) AS INT)) AS `countofhost`'>, Column<b'sum(CAST((timestamp IS NULL) AS INT)) AS `countoftimestamp`'>, Column<b'sum(CAST((method IS NULL) AS INT)) AS `countofmethod`'>, Column<b'sum(CAST((endpoint IS NULL) AS INT)) AS `countofendpoint`'>, Column<b'sum(CAST((status IS NULL) AS INT)) AS `countofstatus`'>, Column<b'sum(CAST((content_size IS NULL) AS INT)) AS `countofcontent_size`'>]
+-----------+----------------+-------------+---------------+-------------+-------------------+
|countofhost|countoftimestamp|countofmethod|countofendpoint|countofstatus|countofcontent_size|
+-----------+----------------+-------------+---------------+-------------+-------------------+
|          0|               0|            0|              0|        60032|              66366|
+-----------+----------------+-------------+---------------+-------------+-------------------+



#### (Describe in sql abd  DF to RDD and Get columns) or Use df.columns

In [12]:


# Generate the SQL statement dynamically
sql = "SELECT " + ", ".join([f"COUNT(IF({col} IS NULL, 1, NULL)) AS count_{col}" for col in logs_df.columns]) + f" FROM {'logs'}"

# Execute the SQL statement
result = spark.sql(sql)

# Display the result
result.show()


+----------+---------------+------------+--------------+------------+------------------+
|count_host|count_timestamp|count_method|count_endpoint|count_status|count_content_size|
+----------+---------------+------------+--------------+------------+------------------+
|         0|              0|           0|             0|       60032|             66366|
+----------+---------------+------------+--------------+------------+------------------+




#### 5. Replace null values with constants such as 0 

In [13]:
from pyspark.sql.functions import when

#changed only these two column as they only have null values

logs_df1=logs_df.withColumn('status' , when(logs_df['status'].isNull(), 0 ).otherwise(logs_df['status']))

logs_df1=logs_df1.withColumn('content_size' , when(logs_df['content_size'].isNull(), 0 ).otherwise(logs_df1['content_size']))

logs_df1.show(5)

+---------------+--------------------+------+--------------------+------+------------+
|           host|           timestamp|method|            endpoint|status|content_size|
+---------------+--------------------+------+--------------------+------+------------+
|109.169.248.247|12/Dec/2015:18:25...|   GET|     /administrator/|   200|        4263|
|109.169.248.247|12/Dec/2015:18:25...|  POST|/administrator/in...|   200|        4494|
|    46.72.177.4|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
|    46.72.177.4|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|
| 83.167.113.100|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
+---------------+--------------------+------+--------------------+------+------------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import col,sum as spark_sum
logs_df1.agg(*[spark_sum((col(c)==0).cast("integer")).alias(f"count_{c}") for c in logs_df1.columns]).show()


+----------+---------------+------------+--------------+------------+------------------+
|count_host|count_timestamp|count_method|count_endpoint|count_status|count_content_size|
+----------+---------------+------------+--------------+------------+------------------+
|      null|           null|        null|          null|       60032|             66690|
+----------+---------------+------------+--------------+------------+------------------+



In [15]:
logs_df2=logs_df.na.fill(0)
logs_df2.show(5)

+---------------+--------------------+------+--------------------+------+------------+
|           host|           timestamp|method|            endpoint|status|content_size|
+---------------+--------------------+------+--------------------+------+------------+
|109.169.248.247|12/Dec/2015:18:25...|   GET|     /administrator/|   200|        4263|
|109.169.248.247|12/Dec/2015:18:25...|  POST|/administrator/in...|   200|        4494|
|    46.72.177.4|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
|    46.72.177.4|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|
| 83.167.113.100|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
+---------------+--------------------+------+--------------------+------+------------+
only showing top 5 rows



In [16]:
logs_df2.agg(*[spark_sum((col(c)==0).cast("integer")).alias(f"count_{c}") for c in logs_df2.columns]).show()

+----------+---------------+------------+--------------+------------+------------------+
|count_host|count_timestamp|count_method|count_endpoint|count_status|count_content_size|
+----------+---------------+------------+--------------+------------+------------------+
|      null|           null|        null|          null|       60032|             66690|
+----------+---------------+------------+--------------+------------+------------------+



In [17]:
# Generate the SQL statement dynamically
select_expr = ", ".join([f"COALESCE({col}, 0) AS {col}" for col in logs_df1.columns])
sql = f"SELECT {select_expr} FROM {'logs'}"

# Execute the SQL statement
result = spark.sql(sql)

# Show the result
result.show(5)

+---------------+--------------------+------+--------------------+------+------------+
|           host|           timestamp|method|            endpoint|status|content_size|
+---------------+--------------------+------+--------------------+------+------------+
|109.169.248.247|12/Dec/2015:18:25...|   GET|     /administrator/|   200|        4263|
|109.169.248.247|12/Dec/2015:18:25...|  POST|/administrator/in...|   200|        4494|
|    46.72.177.4|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
|    46.72.177.4|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|
| 83.167.113.100|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|
| 83.167.113.100|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|
|   95.29.198.15|12/Dec/2015:18:32...|   GET|     /administrator/|   200|        4263|
|   95.29.198.15|12/Dec/2015:18:32...|  POST|/administrator/in...|   200|        4494|
|  109.184.11.34|12/Dec/2015:18:32...|   GE

In [18]:

sql="SELECT " + ", ".join(["SUM(CASE WHEN {} = 0 THEN 1 ELSE 0 END) AS count_{}".format(col, col) for col in logs_df2.columns]) + " FROM {}".format('logs')


spark.sql(sql).show(5)


+----------+---------------+------------+--------------+------------+------------------+
|count_host|count_timestamp|count_method|count_endpoint|count_status|count_content_size|
+----------+---------------+------------+--------------+------------+------------------+
|         0|              0|           0|             0|           0|               324|
+----------+---------------+------------+--------------+------------+------------------+



#### 6.Parse timestamp to readable date.

In [19]:
from pyspark.sql.functions import to_timestamp, date_format

In [20]:
logs_df1 = logs_df1.withColumn('date', date_format(to_timestamp('timestamp', 'dd/MMM/yyyy:HH:mm:ss'), 'yyyy-MM-dd'))
logs_df1.show(5)

+---------------+--------------------+------+--------------------+------+------------+----------+
|           host|           timestamp|method|            endpoint|status|content_size|      date|
+---------------+--------------------+------+--------------------+------+------------+----------+
|109.169.248.247|12/Dec/2015:18:25...|   GET|     /administrator/|   200|        4263|2015-12-12|
|109.169.248.247|12/Dec/2015:18:25...|  POST|/administrator/in...|   200|        4494|2015-12-12|
|    46.72.177.4|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|2015-12-12|
|    46.72.177.4|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|2015-12-12|
| 83.167.113.100|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|2015-12-12|
+---------------+--------------------+------+--------------------+------+------------+----------+
only showing top 5 rows



In [25]:
spark.sql("""SELECT *, date_format(to_timestamp(timestamp, 'dd/MMM/yyyy:HH:mm:ss'), 'yyyy-MM-dd') AS date FROM logs """).show(5)

+---------------+--------------------+------+--------------------+------+------------+----------+
|           host|           timestamp|method|            endpoint|status|content_size|      date|
+---------------+--------------------+------+--------------------+------+------------+----------+
|109.169.248.247|12/Dec/2015:18:25...|   GET|     /administrator/|   200|        4263|2015-12-12|
|109.169.248.247|12/Dec/2015:18:25...|  POST|/administrator/in...|   200|        4494|2015-12-12|
|    46.72.177.4|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|2015-12-12|
|    46.72.177.4|12/Dec/2015:18:31...|  POST|/administrator/in...|   200|        4494|2015-12-12|
| 83.167.113.100|12/Dec/2015:18:31...|   GET|     /administrator/|   200|        4263|2015-12-12|
+---------------+--------------------+------+--------------------+------+------------+----------+
only showing top 5 rows



#### 7. Describe which HTTP status values appear in data and how many. 

In [30]:
spark.sql("select status,count(status) from logs group by status ").show()

+------+-------------+
|status|count(status)|
+------+-------------+
|   412|           19|
|   406|           53|
|  null|            0|
|   206|       939908|
|   500|         3225|
|   301|          609|
|   400|           20|
|   403|         2217|
|   404|       186787|
|   200|      1138403|
|   303|          244|
|   304|         6329|
|   405|            7|
|   401|          153|
+------+-------------+



In [38]:
spark.sql("select status,count(status) over(PARTITION BY status) from logs ").distinct().show()

+------+-------------------------------------------------------------------------------------------------+
|status|count(status) OVER (PARTITION BY status ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)|
+------+-------------------------------------------------------------------------------------------------+
|   406|                                                                                               53|
|   412|                                                                                               19|
|  null|                                                                                                0|
|   206|                                                                                           939908|
|   500|                                                                                             3225|
|   301|                                                                                              609|
|   400|                             

In [35]:
from pyspark.sql.functions import count
from pyspark.sql.window import Window
window_spec = Window.partitionBy('status')

# Use window functions to count occurrences of each status
logs_df1.select(col('status'), count(col('status')).over(window_spec).alias('count')).distinct().show()

+------+-------+
|status|  count|
+------+-------+
|   406|     53|
|   412|     19|
|   206| 939908|
|   500|   3225|
|   301|    609|
|   400|     20|
|   403|   2217|
|   404| 186787|
|   200|1138403|
|   303|    244|
|   304|   6329|
|   405|      7|
|     0|  60032|
|   401|    153|
+------+-------+



In [47]:
logs_df1.groupBy('status').count().show()

+------+-------+
|status|  count|
+------+-------+
|   412|     19|
|   406|     53|
|   206| 939908|
|   500|   3225|
|   301|    609|
|   400|     20|
|   403|   2217|
|   404| 186787|
|   200|1138403|
|   303|    244|
|   304|   6329|
|   405|      7|
|     0|  60032|
|   401|    153|
+------+-------+



#### 8. How many unique hosts are there in the entire log and their average request 

In [49]:
spark.sql("select host,count(host) from logs group by host").show(5)

+-------------+-----------+
|         host|count(host)|
+-------------+-----------+
|  46.72.177.4|          8|
|194.48.218.78|          2|
|31.181.253.16|          2|
| 37.112.46.76|          2|
|95.107.90.225|          2|
+-------------+-----------+
only showing top 5 rows



In [54]:
spark.sql("select count(host)/count(distinct host) from logs ").show(5)

+--------------------------------------------------------------------+
|(CAST(count(host) AS DOUBLE) / CAST(count(DISTINCT host) AS DOUBLE))|
+--------------------------------------------------------------------+
|                                                   57.25355078851993|
+--------------------------------------------------------------------+



In [62]:
from pyspark.sql.functions import countDistinct
logs_df1.select(count('host')/countDistinct('host')).show()

+------------------------------------+
|(count(host) / count(DISTINCT host))|
+------------------------------------+
|                   57.25355078851993|
+------------------------------------+

